# Loan and housing data —loading and basic structuring

---
por [<span style="font-family:monospace;">Andres Ethorimn</span>](https://www.linkedin.com/in/ethlob/) 👨🏾‍💻

<span style="color:green; font-weight:bold;">Staging:</span> en este notebook vamos a recolectar la información suministrada para este estudio, aplicar técnicas de trasformación báscias y realizar una estrucutración de los datos para su posterior análisis.

<span style="font-size:small; color:gray;">*Los notebooks en esta sección son prototipos. Su uso es exclusivo para exploración y aprendizaje, y no están pensado para usarse en producción. Tenga presente que los comentarios de aquí en adelante estarán en inglés por ser el idioma estándar de la industria.*</span>


In [2]:
# Import necessary libraries
import os
import pandas as pd

In [3]:
# Define useful paths and data directories
parent_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(parent_dir, 'data')

In [3]:
# Extracting column names from the Excel file. We are doing this because of the way the data is structured in the file. However, normal data sources respecting the governance of data should not require this step.
names = pd.read_excel(os.path.join(data_dir, 'raw', 'data_prueba_ds_semisenior.xlsx'), nrows=0).columns
names = names.str.split(',').tolist()[0]

# Setting a convention for the first column name as it may be used as an index
names[0] = 'idx'

In [4]:
# We do a similar operation to the data extraction operation. I stringly recommned to not use this approach in production code, as it is not robust and could lead to data integrity issues. A great workaround is to user a data orchestration tool such as Apache Airflow, Prefect, or Databricks to automate ETL processes.
data = pd.read_excel(os.path.join(data_dir, 'raw', 'data_prueba_ds_semisenior.xlsx'), header=None, skiprows=1)
data = pd.DataFrame(data.apply(lambda x: x.str.split(','), axis=1).loc[:, 0].tolist(), columns=names)
data = data.replace(['', ' ', 'null', 'NULL', 'None'], pd.NA)

# Set the index of the DataFrame to the first column we called
data.set_index('idx', inplace=True)

In [5]:
# Select the numeric columns according to the problem description and the provided data dictionary.
numeric_columns = [
    'age', 'default', 'balance', 'housing', 'loan', 'duration', 'campaign', 'pdays', 'previous', 'deposit', 'tenencia_ahorros', 'tenencia_corriente', 'tenencia_cdt', 'tenencia_tdc', 'tenencia_lb', 'tenencia_vehiculo', 'ingresos', 'egresos', 'saldo_captacion', 'saldo_tdc', 'monto_trx_tdc', 'cantidad_trx_tdc', 'saldo_lb', 'saldo_veh', 'monto_trx_debito', 'cantidad_trx_debito'
]

# Reassigning types for the numeric columns
data.loc[:, numeric_columns] = data.loc[:, numeric_columns].apply(pd.to_numeric, errors='coerce', downcast='integer', axis=0)

In [ ]:
# Saving data to a new Excel file in the staging directory
data.to_excel(os.path.join(data_dir, 'staging', '01_data_prueba_ds_loan_housing_basic_schema.xlsx'), index=True, engine='openpyxl')